In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf
import math
import os
import sys
# inference_so_dir = os.path.join("C:\\Users\\Public\\Documents\\Konrad\\zindi\\women_poverty\\venv\\Lib\\site-packages\\tensorflow_decision_forests\\tensorflow\\ops\\inference")
#
# os.environ['PATH'] = inference_so_dir + os.pathsep + os.environ['PATH']
import tensorflow_decision_forests as tfdf
from sklearn.model_selection import train_test_split
from wurlitzer import sys_pipes
from IPython.core.magic import register_line_magic
from IPython.display import Javascript

%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 10]

NotFoundError: C:\Users\Public\Documents\Konrad\zindi\women_poverty\venv\Lib\site-packages\tensorflow_decision_forests\tensorflow\ops\inference\inference.so not found

## Load Data

In [1]:
df = pd.read_csv('outputs/Train.csv')
test_df = pd.read_csv('outputs/Test.csv')
vocab_df = pd.read_csv('variable_descriptions.csv')
admin_df = pd.read_csv('zaf_adminboundaries_tabulardata.csv', sep=";")

admin_df["AREA_SQKM"] = admin_df["AREA_SQKM"].str.replace(",", ".").astype(float)
df = pd.merge(df, admin_df, on="ADM4_PCODE", how="left")
test_df = pd.merge(test_df, admin_df, on="ADM4_PCODE", how="left")

X = df.drop("target", axis=1)
y = df["target"]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

NameError: name 'pd' is not defined

In [ ]:
train_ds_pd = pd.concat([X_train, y_train], axis=1)
test_ds_pd = pd.concat([X_val, y_val], axis=1)
label = "target"
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)

In [ ]:
# Specify the model.
model_1 = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)

# Optionally, add evaluation metrics.
model_1.compile(metrics=["mse"])

# Train the model.
# "sys_pipes" is optional. It enables the display of the training logs.
# TODO
with sys_pipes():
    model_1.fit(x=train_ds)

evaluation = model_1.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
    print(f"{name}: {value:.4f}")